# Preprocesamiento y modelado de datos de viviendas

## Importación de librerías y carga de datos

In [1]:
import pandas as pd
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler

In [2]:
df_inicial = pd.read_csv('../data/final/viviendas_2011_2024.csv')
df_inicial.head()

,Ano,Distrito,Esperanza_vida,Renta_neta_persona,Renta_neta_hogar,Renta_bruta_persona,Renta_bruta_hogar,Edad_media,Mayores_65anos%,Menores_18anos%,...,Terraza,Planta,Exterior,Ascensor,Ano_construccion,Ano_reforma,Tipo_vivienda,Banos,Precio_predicho,Precio_ajustado
0,2011,CENTRO,83.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,0.0,False,False,NaN,NaN,apartamento,1,672.875811,657.471490
1,2011,CENTRO,83.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,5.0,True,True,NaN,NaN,apartamento,1,923.555035,894.792822
2,2011,CENTRO,83.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,3.0,True,True,1910.0,NaN,apartamento,2,1424.913483,1434.552312
3,2011,CENTRO,83.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,4.0,True,True,NaN,NaN,apartamento,1,2240.808388,2335.359589
4,2011,CENTRO,83.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,3.0,True,True,1940.0,NaN,apartamento,1,1583.237203,1571.481771


## Preprocesamiento

Revisar notebook [EDA_viviendas_2011_2024](./EDA_viviendas_2011_2024.ipynb) para visualizar el análisis que da lugar a las decisiones tomadas en el preprocesamiento.

### 1. Eliminar e imputar columnas

Eliminación de variables fuertemente correlacionadas, de variables con más del 90% de nulos e imputación de nulos de la columna "Planta".

In [3]:
df = df_inicial.copy()

# Elminación de Ano_construccion y Ano_reforma por exceso de nulos
df = df.drop(columns=['Ano_construccion', 'Ano_reforma'])

# Crear columna de indicador de nulos para Planta
df['Planta_is_missing'] = df['Planta'].isna().astype(int)

# Imputar valores nulos con la mediana para Planta (por distrito)
df['Planta'] = df.groupby('Distrito')['Planta'].transform(lambda x: x.fillna(x.median()))

# Variables muy correlacionadas y no usadas
var_corr = ['Tamano_vivienda_personas', 'Superficie_distrito_ha', 'Edad_media', 'Renta_neta_persona',
            'Renta_neta_hogar', 'Renta_bruta_persona', 'Renta_bruta_hogar', 'Precio_predicho']

df = df.drop(columns=var_corr)

print('Columnas restantes:', df.columns.to_list())

Columnas restantes: ['Ano', 'Distrito', 'Esperanza_vida', 'Mayores_65anos%', 'Menores_18anos%', 'Paro_registrado%', 'Apartamentos_turisticos', 'Densidad_poblacion', 'Zonas_verdes%', 'Habitaciones', 'Operacion', 'Tamano', 'Garaje', 'Trastero', 'Piscina', 'Terraza', 'Planta', 'Exterior', 'Ascensor', 'Tipo_vivienda', 'Banos', 'Precio_ajustado', 'Planta_is_missing']


### 2. Transformar variables categóricas

Se deben pasar las variables categóricas a numéricas para poder procesarlas correctamente con los modelos. Las tres variables categóricas son `Distrito`, `Operacion` y `Tipo_vivienda`. Como no son variables que tengan jerarquía, usaremos dummies para transformarlas.

In [9]:
df = pd.get_dummies(df, drop_first=True)

print(df.columns)

Index(['Ano', 'Esperanza_vida', 'Mayores_65anos%', 'Menores_18anos%',
       'Paro_registrado%', 'Apartamentos_turisticos', 'Densidad_poblacion',
       'Zonas_verdes%', 'Habitaciones', 'Tamano', 'Garaje', 'Trastero',
       'Piscina', 'Terraza', 'Planta', 'Exterior', 'Ascensor', 'Banos',
       'Precio_ajustado', 'Planta_is_missing', 'Distrito_BARAJAS',
       'Distrito_CARABANCHEL', 'Distrito_CENTRO', 'Distrito_CHAMARTIN',
       'Distrito_CHAMBERI', 'Distrito_CIUDADLINEAL',
       'Distrito_FUENCARRALELPARDO', 'Distrito_HORTALEZA', 'Distrito_LATINA',
       'Distrito_MONCLOAARAVACA', 'Distrito_MORATALAZ',
       'Distrito_PUENTEDEVALLECAS', 'Distrito_RETIRO', 'Distrito_SALAMANCA',
       'Distrito_SANBLASCANILLEJAS', 'Distrito_TETUAN', 'Distrito_USERA',
       'Distrito_VICALVARO', 'Distrito_VILLADEVALLECAS', 'Distrito_VILLAVERDE',
       'Operacion_venta', 'Tipo_vivienda_chalet', 'Tipo_vivienda_dúplex',
       'Tipo_vivienda_estudio', 'Tipo_vivienda_loft', 'Tipo_vivienda_mansión',


### 3. Dividir en entrenamiento, validación y test

**DECIDIR Y MODIFICAR - No se puede usar 2023 y 2024 para test o validación porque no tenemos variables input! Solo se pueden usar esos años para evaluar manualmente.**

Escogeremos años de 2015 a 2022 (ambos inclusive) para entrenamiento; año 2023 para validación; y año 2024 para test. Como se trata de una serie temporal, haremos la separación directamente en base a los años.

In [ ]:
train_mask = (df['Ano'] >= 2015) & (df['Ano'] <= 2022)
val_mask = df['Ano'] == 2023
test_mask = df['Ano'] == 2024

### 3. Preparar un pipeline de datos

Se utilizará un pipeline para facilitar el manejo de datos y evitar fugas. Para ello, elegimos un escalado robusto para procesar mejor los outliers; y un modelo para entrenar.

In [12]:
def definir_pipeline(modelo) -> Pipeline:
    """
    Función para aplicar escalado y decidir el modelo que se usará.
    
    Parameters
    ----------
    modelo : KNN, XGBoost, MLP...
        Se pueden incluir los parámetros manualmente dentro del modelo.
    
    Returns
    ----------
        pipeline
    """

    pipeline = Pipeline([
        ('scaler', RobustScaler()),
        ('modelo', modelo)
    ])

    return pipeline

**Faltaría a partir de aquí elegir modelos (recomendado por lo menos: baseline con un modelo linear o de distancias; un modelo de árboles; y un MLP). Se puede hacer optimización inteligente con Optuna. Validación para hacer pruebas, test cuando ya se haya optimizado lo posible.**